<a href="https://colab.research.google.com/github/Sai1099/App-Analysis/blob/main/bajaj_finserv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install findspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=32cdcbacd25541f6750f4a654ed62dabe6ead9c7c3a2bb330d93b7f3c9fab611
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql.types import StructField,StructType,IntegerType,StringType

In [7]:
from pyspark.sql.functions import *

In [8]:
spark = SparkSession.builder.appName("Bajaj-finserv").getOrCreate()

In [9]:
!pip install spacy

In [10]:
import spacy

In [11]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [12]:
nlp = spacy.load("en_core_web_sm")


In [13]:
excluded_words = spacy.lang.en.stop_words.STOP_WORDS

In [14]:
print(excluded_words)

{'such', 'whom', 'less', 'everything', 'those', 'using', 'back', 'after', 'has', 'whence', 'its', 'hence', "n't", 'fifteen', 'she', 'twelve', 'nothing', 'go', 'behind', '’re', 'around', 'throughout', 'beforehand', 'how', 'amongst', 'since', 'one', 'does', 'much', 'doing', 'keep', 'otherwise', 'seem', 'upon', 'had', 'somehow', 'do', 'either', 'various', 'sometimes', 'ourselves', 'which', 'if', 'both', 'made', 'into', 'his', "'m", 'never', 'empty', 'latter', 'yourself', 'where', 'next', 'something', 'for', 'three', 'few', 'whenever', 'whereafter', 'though', 'already', 'serious', 'when', 'off', 'thereby', '’ll', 'via', 'others', 'them', 'under', 'third', 'nor', 'everywhere', 'an', 'still', 're', 'n’t', 'whereupon', 'indeed', 'against', 'more', 'herein', 'then', 'are', "'s", 'any', 'not', 'it', 'whose', 'even', 'hereupon', 'noone', 'beside', 'so', 'your', 'am', 'within', 'side', 'of', 'sometime', 'became', 'i', 'someone', 'get', 'at', 'itself', 'forty', 'hereby', 'above', 'too', 'hers', 'o

In [15]:
schema = StructType([ \
    StructField("Name",StringType(),True), \
    StructField("Rating",IntegerType(),True), \
    StructField("Review",StringType(),True), \
    StructField("Date",StringType(),True), \
    StructField("Useful",IntegerType(),True), \
    StructField("Answered",StringType(),True)
    ])

In [16]:
main_df = spark.read.csv('/content/reviews.txt', schema=schema, sep=']')


In [17]:
main_df = main_df.withColumn("Useful",
                             when(col("Useful").isNull(), 0)
                             .otherwise(col("Useful")))

In [20]:
main_df.show(40)

+--------------------+------+--------------------+----+------+--------+
|                Name|Rating|              Review|Date|Useful|Answered|
+--------------------+------+--------------------+----+------+--------+
|      Kunga Wangchuk|     5|Very easy to use ...|NULL|     0|     Yes|
|       Ayush Chauhan|     5|1 really like thi...|NULL|    38|     Yes|
|             Kamlesh|     1|I don't like the ...|NULL|     3|     Yes|
|       Rahul Waghela|     1|If I could give 0...|NULL|   154|     Yes|
|          Vishal Raj|     1|Such a pathatic u...|NULL|     0|     Yes|
|        Neeraj Singh|     1|Worst app! They h...|NULL|     3|     Yes|
| ANSHUMAN Nahar jain|     5|Recently started ...|NULL|   174|     Yes|
|             Nagaraj|     1|If you don't know...|NULL|    86|     Yes|
|Naqeeb Nasrulla Khan|     1|The worst app eve...|NULL|    74|     Yes|
|         Amir Mondal|     4|Stuck at MPin cre...|NULL|    11|     Yes|
|      Sandeep Gavali|     5|I have been using...|NULL|   243|  

In [22]:
main_df.createOrReplaceTempView("bajaj")

In [23]:
cleaned = spark.sql("SELECT Name,Review,Useful FROM bajaj;")

In [24]:
cleaned.show(truncate=False)

+-------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Name                     |Review                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [25]:
words = main_df.select(explode(split(col("Review")," ")).alias("word"))

In [26]:
words_finder = words.groupBy("word").agg(count("word").alias("count")).orderBy(desc("count"))

In [27]:
filtered_words = words_finder.filter(~col("word").isin(excluded_words))

In [28]:
filtered_words.show(60)

+-----------+-----+
|       word|count|
+-----------+-----+
|          I|  788|
|        app|  707|
|        The|  259|
|       loan|  247|
|       easy|  219|
|      Bajaj|  213|
|       card|  207|
|        use|  156|
|   customer|  155|
|       This|  150|
|       time|  148|
|       it's|  138|
| experience|  134|
|  interface|  123|
|        emi|  117|
|       Very|  117|
|  financial|  114|
|       app.|  113|
|         It|  112|
|application|  112|
|       good|  108|
|        EMI|  102|
|          .|   96|
|    service|   95|
|      don't|   90|
|       user|   88|
|      loans|   86|
|       like|   85|
|        UPI|   85|
|    Finserv|   84|
|       It's|   80|
|    payment|   79|
|      money|   75|
|     credit|   71|
|   managing|   69|
|      bajaj|   69|
|     update|   69|
|   features|   69|
|    process|   66|
|    showing|   65|
|     option|   64|
|    support|   62|
|     offers|   62|
|       They|   62|
|        pay|   61|
|      Worst|   60|
|   payments|   60|


In [29]:
mainde = spark.sql("SELECT * FROM bajaj WHERE Rating < 4 AND (Review LIKE '%worst%' OR Review LIKE '%Worst%');")

In [30]:
mainde.select(count("Review")).show()

+-------------+
|count(Review)|
+-------------+
|          114|
+-------------+



### END



In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="siebert/sentiment-roberta-large-english")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

In [ ]:
inputs = tokenizer("Hello, you are waste", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.logits

In [ ]:
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.2294,  2.4462]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
